<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/mqqt_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import paho.mqtt.client as mqtt
import json
import random
import time
import queue
import threading

# MQTT Configuration
MQTT_BROKER = "test.mosquitto.org"
MQTT_TOPIC = "braude/D106/indoor"
MQTT_PORT = 1883

# Thread-safe queue for storing messages (for MQTT mode)
data_queue = queue.Queue()

def _simulate_data_stream(delay_seconds=2):
    """
    Generator that yields simulated sensor data indefinitely.

    Args:
        delay_seconds (int): Delay between data points in seconds.
    Yields:
        dict: Simulated sensor data.
    """
    while True:
        simulated_data = {
            "Temperature": round(random.uniform(22, 28), 2),
            "Humidity": round(random.uniform(40, 70), 2),
            "Pressure": round(random.uniform(1005, 1020), 2)
        }
        yield simulated_data
        time.sleep(delay_seconds)

def _on_mqtt_message(client, userdata, msg):
    """
    Callback for incoming MQTT messages.

    Parses the payload and adds the result to the data queue.
    """
    try:
        data = json.loads(msg.payload.decode())
        data_queue.put(data)
    except json.JSONDecodeError:
        pass  # Ignore invalid messages

def _start_mqtt_listener():
    """
    Starts a background thread that listens for MQTT messages.
    """
    client = mqtt.Client()
    client.on_message = _on_mqtt_message
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.subscribe(MQTT_TOPIC)

    thread = threading.Thread(target=client.loop_forever, daemon=True)
    thread.start()

def _mqtt_data_stream():
    """
    Generator that yields sensor data received via MQTT.

    Yields:
        dict: Real sensor data as received from the MQTT broker.
    """
    _start_mqtt_listener()
    while True:
        data = data_queue.get()  # Waits until data is available
        yield data

def get_live_data_stream(mode="simulation"):
    """
    Returns a generator that yields sensor data indefinitely.

    Args:
        mode (str): Either 'simulation' or 'mqtt'.

    Returns:
        generator: A generator yielding sensor data dictionaries.
    """
    if mode == "simulation":
        return _simulate_data_stream()
    elif mode == "mqtt":
        return _mqtt_data_stream()
    else:
        raise ValueError("Invalid mode. Use 'simulation' or 'mqtt'.")

Running in SIMULATION mode...

Temperature: 26.75°C
Humidity: 51.28%
Pressure: 1010.97 hPa
------------------------------
Temperature: 22.73°C
Humidity: 67.02%
Pressure: 1006.22 hPa
------------------------------
Temperature: 22.2°C
Humidity: 58.23%
Pressure: 1006.23 hPa
------------------------------
Temperature: 27.07°C
Humidity: 44.24%
Pressure: 1005.03 hPa
------------------------------
Temperature: 24.14°C
Humidity: 63.65%
Pressure: 1007.54 hPa
------------------------------
Temperature: 26.96°C
Humidity: 62.58%
Pressure: 1009.1 hPa
------------------------------
Temperature: 27.11°C
Humidity: 54.96%
Pressure: 1006.58 hPa
------------------------------
Temperature: 27.04°C
Humidity: 51.49%
Pressure: 1010.88 hPa
------------------------------
Temperature: 25.68°C
Humidity: 57.88%
Pressure: 1010.63 hPa
------------------------------
Temperature: 22.76°C
Humidity: 68.12%
Pressure: 1011.34 hPa
------------------------------


KeyboardInterrupt: 